In [1]:
pip install tensorflow-addons==0.11.1

     |████████████████████████████████| 1.1MB 4.5MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


In [2]:
import os
import tensorflow as tf


def regularized_padded_conv(*args, **kwargs):
    return tf.keras.layers.Conv2D(*args, **kwargs, padding='same', kernel_regularizer=_regularizer,
                                  kernel_initializer='he_normal', use_bias=False)


def BN_ReLU(x):
    x = tf.keras.layers.BatchNormalization()(x)
    return tf.keras.layers.ReLU()(x)


def shortcut(x, filters, stride, mode):
    if x.shape[-1] == filters:
        return x
    elif mode == 'B':
        return regularized_padded_conv(filters, 1, strides=stride)(x)
    elif mode == 'B_original':
        x = regularized_padded_conv(filters, 1, strides=stride)(x)
        return tf.keras.layers.BatchNormalization()(x)
    elif mode == 'A':
        return tf.pad(tf.keras.layers.MaxPool2D(1, stride)(x) if stride>1 else x,
                      paddings=[(0, 0), (0, 0), (0, 0), (0, filters - x.shape[-1])])
    else:
        raise KeyError("Parameter shortcut_type not recognized!")
    

def original_block(x, filters, stride=1, **kwargs):
    c1 = regularized_padded_conv(filters, 3, strides=stride)(x)
    c2 = regularized_padded_conv(filters, 3)(BN_ReLU(c1))
    c2 = tf.keras.layers.BatchNormalization()(c2)
    
    mode = 'B_original' if _shortcut_type == 'B' else _shortcut_type
    x = shortcut(x, filters, stride, mode=mode)
    return tf.keras.layers.ReLU()(x + c2)
    
    
def preactivation_block(x, filters, stride=1, preact_block=False):
    flow = BN_ReLU(x)
    if preact_block:
        x = flow
        
    c1 = regularized_padded_conv(filters, 3, strides=stride)(flow)
    if _dropout:
        c1 = tf.keras.layers.Dropout(_dropout)(c1)
        
    c2 = regularized_padded_conv(filters, 3)(BN_ReLU(c1))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c2


def bootleneck_block(x, filters, stride=1, preact_block=False):
    flow = BN_ReLU(x)
    if preact_block:
        x = flow
         
    c1 = regularized_padded_conv(filters//_bootleneck_width, 1)(flow)
    c2 = regularized_padded_conv(filters//_bootleneck_width, 3, strides=stride)(BN_ReLU(c1))
    c3 = regularized_padded_conv(filters, 1)(BN_ReLU(c2))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c3


def group_of_blocks(x, block_type, num_blocks, filters, stride, block_idx=0):
    global _preact_shortcuts
    preact_block = True if _preact_shortcuts or block_idx == 0 else False
    
    x = block_type(x, filters, stride, preact_block=preact_block)
    for i in range(num_blocks-1):
        x = block_type(x, filters)
    return x


def Resnet(input_shape, n_classes, l2_reg=1e-4, group_sizes=(2, 2, 2), features=(16, 32, 64), strides=(1, 2, 2),
           shortcut_type='B', block_type='preactivated', first_conv={"filters": 16, "kernel_size": 3, "strides": 1},
           dropout=0, cardinality=1, bootleneck_width=4, preact_shortcuts=True):
    
    global _regularizer, _shortcut_type, _preact_projection, _dropout, _cardinality, _bootleneck_width, _preact_shortcuts
    _bootleneck_width = bootleneck_width # used in ResNeXts and bootleneck blocks
    _regularizer = tf.keras.regularizers.l2(l2_reg)
    _shortcut_type = shortcut_type # used in blocks
    _cardinality = cardinality # used in ResNeXts
    _dropout = dropout # used in Wide ResNets
    _preact_shortcuts = preact_shortcuts
    
    block_types = {'preactivated': preactivation_block,
                   'bootleneck': bootleneck_block,
                   'original': original_block}
    
    selected_block = block_types[block_type]
    inputs = tf.keras.layers.Input(shape=input_shape)
    flow = regularized_padded_conv(**first_conv)(inputs)
    
    if block_type == 'original':
        flow = BN_ReLU(flow)
    
    for block_idx, (group_size, feature, stride) in enumerate(zip(group_sizes, features, strides)):
        flow = group_of_blocks(flow,
                               block_type=selected_block,
                               num_blocks=group_size,
                               block_idx=block_idx,
                               filters=feature,
                               stride=stride)
    
    if block_type != 'original':
        flow = BN_ReLU(flow)
    
    flow = tf.keras.layers.GlobalAveragePooling2D()(flow)
    outputs = tf.keras.layers.Dense(n_classes, kernel_regularizer=_regularizer)(flow)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def load_weights_func(model, model_name):
    try: model.load_weights(os.path.join('saved_models', model_name + '.tf'))
    except tf.errors.NotFoundError: print("No weights found for this model!")
    return model


def cifar_wide_resnet(N, K, block_type='preactivated', shortcut_type='B', dropout=0, l2_reg=2.5e-4):
    assert (N-4) % 6 == 0, "N-4 has to be divisible by 6"
    lpb = (N-4) // 6 # layers per block - since N is total number of convolutional layers in Wide ResNet
    model = Resnet(input_shape=(32, 32, 3), n_classes=10, l2_reg=l2_reg, group_sizes=(lpb, lpb, lpb), features=(16*K, 32*K, 64*K),
                   strides=(1, 2, 2), first_conv={"filters": 16, "kernel_size": 3, "strides": 1}, shortcut_type=shortcut_type,
                   block_type=block_type, dropout=dropout, preact_shortcuts=True)
    return model


def WRN_28_2(shortcut_type='B', load_weights=False, dropout=0, l2_reg=2.5e-4):
    model = cifar_wide_resnet(28, 2, 'preactivated', shortcut_type, dropout=dropout, l2_reg=l2_reg)
    if load_weights: model = load_weights_func(model, 'cifar_WRN_28_10')
    return model


def FixMatch():
  self.resnet = WRN_28_2()

  """
  Unlabeled data
  weak_input = weak_transformation(input)
  strong_input = strong_transformation(input)

  weak_output = resnet(weak_input)
  strong_output = resnet(strong_input)
  
  pseudo_label = pseudolabel(weak_output)
  cross_ent = crossentropy(pseudo_label, strong_output)

  """





In [87]:
import tensorflow_probability as tfp

class CTAugment:
  def __init__(self, decay, threshold, depth, n_bins):
    self.TRANSFORMATIONS = [ self.autocontrast, self.blur, self.brightness,
                          self.color, self.contrast, self.cutout, self.equalize,
                          self.invert, self.identity, self.posterize, 
                          self.rescale, self.rotate, self.sharpness, 
                          self.shear_x, self.shear_y, self.smooth, 
                          self.solarize, self.translate_x, self.translate_y]

    self.decay = decay
    self.threshold = threshold
    self.depth = depth
    self.n_bins = n_bins
  
    # we need some way of storing functions so that we can randomly sample
    # from them. The format below might not be perfect but it is nice if we 
    # can generate a index i with which we can access the ith
    # transformation, its corresponding bins and their weights
    self.bins = [[]]
    self.weights = np.ones([len(self.TRANSFORMATIONS), self.n_bins])

  def augment(self, x):
      for k in range(self.depth):
        # we generate a function randomly (here for each sample in batcg)
        i = np.random.randint(0, len(self.TRANSFORMATIONS), size=x.shape[0]) 

        
        # pick weights for correpsonding function and set weigths to 0 if they 
        # are less than 0.8
        logits_i = self.weights[i]
        threshold_indices = logits_i < self.threshold
        logits_i[threshold_indices] = 0

        # are the outputs really logits?

        # We should probably do this for a whole batch at onces if possible
        dist = tfp.distributions.Categorical(logits_i)
        bin = dist.sample(1)

        # we should probably copy here so we do not overwrite original
        x = transformation(x)
      
      return x


  def update_weights(parameter, weights):
      pass



  def autocontrast():
      pass
  
  def blur():
      pass
  
  def brightness():
      pass

  def color():
      pass

  def contrast():
      pass

  def cutout():
      pass

  def equalize():
      pass

  def invert():
      pass
  
  def identity():
      pass

  def posterize():
      pass

  def rescale():
      pass

  def rotate():
      pass

  def sharpness():
      pass

  def shear_x():
      pass

  def shear_y():
      pass

  def smooth():
      pass

  def solarize():
      pass

  def translate_x():
      pass

  def translate_y():
      pass


In [88]:
import os
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
logging.getLogger('tensorflow').disabled = True

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from tqdm import tqdm, tqdm_notebook


def training(model, ds, mean=None, std=None, lr_values=[0.01, 0.1, 0.01, 0.001], lr_boundaries=[400, 32000, 48000, 64000],
                   val_interval=2000, log_interval=200, batch_size=128, nesterov=False):

    schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=lr_boundaries[:-1], values=lr_values)
    optimizer = tf.keras.optimizers.SGD(schedule, momentum=0.9, nesterov=nesterov)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    def train_prep(x, y):
        x = tf.cast(x, tf.float32)/255.
        #x = tf.image.random_flip_left_right(x)
        #x = tf.image.pad_to_bounding_box(x, 4, 4, 40, 40) #must fix ds independent shape
        #x = tf.image.random_crop(x, (32, 32, 3))          #must fix ds independent shape
        if mean is not None and std is not None:
          x = (x - mean) / std
        return x, y

    def valid_prep(x, y):
        x = tf.cast(x, tf.float32)/255.
        if mean is not None and std is not None:
          x = (x - mean) / std
        return x, y

    #ds['train'] = ds['train'].map(train_prep).shuffle(10000).repeat().batch(batch_size).prefetch(-1)
    ds['train'] = ds['train'].map(train_prep).batch(batch_size).prefetch(-1)
    ds['test'] = ds['test'].map(valid_prep).batch(batch_size*4).prefetch(-1)

    #runid = run_name + '_x' + str(np.random.randint(10000))
    #writer = tf.summary.create_file_writer(logdir + '/' + runid)
    accuracy = tf.metrics.SparseCategoricalAccuracy()
    cls_loss = tf.metrics.Mean()
    reg_loss = tf.metrics.Mean()
    
    #print(f"RUNID: {runid}")
    #tf.keras.utils.plot_model(model)#, os.path.join('saved_plots', runid + '.png'))

    def weak_transformation(x):
      x = tf.image.random_flip_left_right(x)
      max_shift = tf.cast(x.shape[1]*0.125, dtype=tf.dtypes.int32)
      shift = tf.random.uniform([x.shape[0], 2], minval=-max_shift, maxval=max_shift, dtype=tf.dtypes.int32)
      return tfa.image.translate(x, tf.cast(shift, tf.dtypes.float32))

    def strong_transformation(x):
      cta = CTAugment(0.99, 0.8, 2, 10)

      x_2 = cta.augment(x)
      pass

    def pseudolabel(class_dist):
        argmax = tf.math.argmax(class_dist, axis=1)
        return tf.one_hot(argmax, class_dist.shape[1])

    def threshold_gate(one_hot, logits, threshold):
        max_probs = tf.math.multiply(one_hot, tf.nn.softmax(logits))
        return tf.cast(max_probs > threshold, max_probs.dtype)# * max_probs

    
    @tf.function
    def step(x, y, training):
        with tf.GradientTape() as tape:
            # unlabeled data
            x_wk = weak_transformation(x)
            outs_wk = model(x_wk, training)  # should this be training or not?
            weak_labels = pseudolabel(outs_wk)
            weak_labels = threshold_gate(weak_labels, outs_wk, 0.7)

            x_str = strong_transformation(x)
            #outs_str = model(x_str, training) #should this be training or not?
            
            #unlabeled_loss = loss_fn(weak_labels, outs_str)
            
            # labeled data
            #outs = model(x, training)
            #labeled_loss = loss_fn(y, outs)

            #add losses together
            #loss = labeled_loss + lambda * unlabeled_loss





            r_loss = tf.add_n(model.losses)
            outs = model(x, training)
            c_loss = loss_fn(y, outs)
            loss = c_loss + r_loss

        if training:
            gradients = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        accuracy(y, outs)
        cls_loss(c_loss)
        reg_loss(r_loss)
        

    training_step = 0
    best_validation_acc = 0
    epochs = lr_boundaries[-1] // val_interval
    
    for epoch in range(epochs):
        for x, y in tqdm(ds['train'].take(val_interval), desc=f'epoch {epoch+1}/{epochs}',
                         total=val_interval, ncols=100, ascii=True):

            training_step += 1
            step(x, y, training=True)

            if training_step % log_interval == 0:
                #with writer.as_default():
                    c_loss, r_loss, err = cls_loss.result(), reg_loss.result(), 1-accuracy.result()
                    print(f" c_loss: {c_loss:^6.3f} | r_loss: {r_loss:^6.3f} | err: {err:^6.3f}", end='\r')

                    tf.summary.scalar('train/error_rate', err, training_step)
                    tf.summary.scalar('train/classification_loss', c_loss, training_step)
                    tf.summary.scalar('train/regularization_loss', r_loss, training_step)
                    tf.summary.scalar('train/learnig_rate', optimizer._decayed_lr('float32'), training_step)
                    cls_loss.reset_states()
                    reg_loss.reset_states()
                    accuracy.reset_states()

        for x, y in ds['test']:
            step(x, y, training=False)

        #with writer.as_default(): TBULATE THE FOLLOWING WHEN UNCOMMENTING!
        tf.summary.scalar('test/classification_loss', cls_loss.result(), step=training_step)
        tf.summary.scalar('test/error_rate', 1-accuracy.result(), step=training_step)
            
        if accuracy.result() > best_validation_acc:
                best_validation_acc = accuracy.result()
                #model.save_weights(os.path.join('saved_models', runid + '.tf'))
                
        cls_loss.reset_states()
        accuracy.reset_states()
            
    
    
def cifar_error_test(model, tr_len=20, vd_len=2):

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.SGD(0.01)

    ds = tfds.load('cifar10', as_supervised=True, in_memory=True)
    std = tf.reshape((0.2023, 0.1994, 0.2010), shape=(1, 1, 3))
    mean= tf.reshape((0.4914, 0.4822, 0.4465), shape=(1, 1, 3))
    
    def train_prep(x, y):
        x = tf.cast(x, tf.float32)/255.
        x = tf.image.random_flip_left_right(x)
        x = tf.image.pad_to_bounding_box(x, 4, 4, 40, 40)
        x = tf.image.random_crop(x, (32, 32, 3))
        x = (x - mean) / std
        return x, y

    def valid_prep(x, y):
        x = tf.cast(x, tf.float32)/255.
        x = (x - mean) / std
        return x, y

    ds['train'] = ds['train'].map(train_prep).batch(5).take(tr_len).prefetch(-1)
    ds['test'] = ds['test'].map(valid_prep).batch(5).take(vd_len).prefetch(-1)

    accuracy = tf.metrics.SparseCategoricalAccuracy()
    cls_loss = tf.metrics.Mean()
    reg_loss = tf.metrics.Mean()

    @tf.function
    def step(x, y, training):
        with tf.GradientTape() as tape:
            r_loss = tf.add_n(model.losses)
            outs = model(x, training)
            c_loss = loss_fn(y, outs)
            loss = c_loss + r_loss

        if training:
            gradients = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        accuracy(y, outs)
        cls_loss(c_loss)
        reg_loss(r_loss)
        
    training_step = 0
    for x, y in tqdm(ds['train'], desc=f'test', total=tr_len, ncols=100, ascii=True):

        training_step += 1
        step(x, y, training=True)
        c_loss, r_loss, err = cls_loss.result(), reg_loss.result(), 1-accuracy.result()
        print(f" c_loss: {c_loss:^6.3f} | r_loss: {r_loss:^6.3f} | err: {err:^6.3f}", end='\r')

    for x, y in ds['test']:
        step(x, y, training=False)

In [89]:
ds = tfds.load('cifar10', as_supervised=True, in_memory=True)

In [90]:
model = WRN_28_2()

In [91]:
training(model, ds)

epoch 1/32:   0%|                                                          | 0/2000 [00:00<?, ?it/s]

(128, 10)
Tensor("Categorical_1/sample/Reshape_2:0", shape=(1, 128), dtype=int32)
(1, 128)


NameError: ignored

In [ ]:
#ds = tfds.load('cifar10', as_supervised=True, in_memory=True)

mean = {
'cifar10': tf.reshape((0.4914, 0.4822, 0.4465), shape=(1, 1, 3)),
'cifar100': tf.reshape((0.5071, 0.4867, 0.4408), shape=(1, 1, 3)),
}

std = {
'cifar10': tf.reshape((0.2023, 0.1994, 0.2010), shape=(1, 1, 3)),
'cifar100': tf.reshape((0.2675, 0.2565, 0.2761), shape=(1, 1, 3))
}